# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip install --upgrade tensorflow keras Pillow

In [33]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout

In [ ]:
tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [59]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [60]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='leaky_relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [61]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [62]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.25))


### Adding a third and a fourth convolutional layer

In [63]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.25))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='leaky_relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.25))



### Step 3 - Flattening

In [64]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [66]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [67]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [68]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [69]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
250/250 [==============================] - 73s 283ms/step - loss: 0.6907 - accuracy: 0.5263 - val_loss: 0.6910 - val_accuracy: 0.5285
Epoch 2/20
250/250 [==============================] - 69s 276ms/step - loss: 0.6772 - accuracy: 0.5645 - val_loss: 0.6328 - val_accuracy: 0.6375
Epoch 3/20
250/250 [==============================] - 67s 267ms/step - loss: 0.6318 - accuracy: 0.6453 - val_loss: 0.5924 - val_accuracy: 0.6900
Epoch 4/20
250/250 [==============================] - 69s 276ms/step - loss: 0.5850 - accuracy: 0.6910 - val_loss: 0.5394 - val_accuracy: 0.7185
Epoch 5/20
250/250 [==============================] - 78s 313ms/step - loss: 0.5643 - accuracy: 0.7080 - val_loss: 0.5195 - val_accuracy: 0.7555
Epoch 6/20
250/250 [==============================] - 62s 246ms/step - loss: 0.5458 - accuracy: 0.7206 - val_loss: 0.5257 - val_accuracy: 0.7370
Epoch 7/20
250/250 [==============================] - 66s 262ms/step - loss: 0.5300 - accuracy: 0.7340 - val_loss: 0.5163 - val_ac

## Part 4 - Making a single prediction

In [49]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [70]:
print(prediction)

dog


## Fim